# What is IForest behavior according to the dataset dimension size?

## Test with 1 dimension dataset

In [1]:
import sys
sys.path.append('../../../../../')
from datasets import datasets as datat
from IForest_DODiMDS import iforest_D as IFD
from metrics import performances
from metrics import visualization
visu = visualization.visualization()

import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from imblearn.metrics import specificity_score
from imblearn.metrics import sensitivity_score
from imblearn.metrics import specificity_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve

import matplotlib.cm as cm
from mpl_toolkits import mplot3d


## Generals settings

In [2]:
max_samples = 256
n_trees = 100
threshold = 0.6

## Chargement du jeu de données

In [3]:
dts = datat.synthetic_datasets(path_top="../../../../../")
X_brut, y_transform, full_dataset = dts.ThreeD_V2()
X_brut.describe()

Columns number = 4


,0,1,2
count,1516.000000,1516.000000,1516.000000
mean,-0.031769,-0.165053,0.171735
std,4.949233,4.933665,7.080026
min,-9.973872,-9.970715,-9.999943
25%,-3.455992,-3.590456,-7.015986
50%,0.004822,-0.084936,0.419743
75%,3.374900,3.326899,7.142706
max,9.977974,9.992620,9.999999


In [4]:
dts.description

"Jeu de données à 2 dimensions constitué de données normales et 1% de données anormales. Les données anormales se retrouvent au centre d'un grand cercle formé par les données normales. Ici, les données anormales sont plus éloignées des données normales mais et se ressemblent beaucoup. \n Données normales : Données aléatoire uniforme, X1 entre -5 et 5, X2 entre -7 et 7, size = 1500, distance euclidienne entre 5 et 7 (Grand cercle de rayon 7 et petit cercle de rayon 5) \n Données anormales : Données aléatoire gaussienne, mu = 0, sigma = 0.5, size = 15."

In [5]:
full_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1516 entries, 0 to 1515
Data columns (total 4 columns):
0    1516 non-null float64
1    1516 non-null float64
2    1516 non-null float64
3    1516 non-null object
dtypes: float64(3), object(1)
memory usage: 47.5+ KB


In [6]:
X_brut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1516 entries, 0 to 1515
Data columns (total 3 columns):
0    1516 non-null float64
1    1516 non-null float64
2    1516 non-null float64
dtypes: float64(3)
memory usage: 35.7 KB


In [7]:
y_transform.describe()

,0
count,1516.000000
mean,0.980211
std,0.198020
min,-1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [8]:
y_transform

,0
0,1
1,1
2,1
3,1
4,1
...,...
1511,-1
1512,-1
1513,-1
1514,-1


## Execution of Iforest to get the best performance

In [9]:
roc_aucs = []
specificities = []
for i in range(10):
    # Instanciation, fit and predict
    func_IFD = IFD.IsolationForest(sample_size = max_samples, n_trees=n_trees)
    func_IFD.fit(X_brut)
    IFD_y_pred_IF, IFD_scores = func_IFD.predict(X=X_brut, threshold=threshold)

    # Matrice de confusion
    ttn, tfp, tfn, ttp = confusion_matrix(y_transform, IFD_y_pred_IF).ravel()
    print(confusion_matrix(y_transform, IFD_y_pred_IF))
    #ROC AUC
    roc_aucs.append(roc_auc_score(y_transform, IFD_y_pred_IF))
    # Specificity
    specificities.append(specificity_score(y_transform, IFD_y_pred_IF))
print("ROC AUC")
print(roc_aucs)
print("Specificities")
print(specificities)
min_roc_auc = max(roc_aucs)
print("min_roc_auc = "+str(min_roc_auc))

[[   0   15]
 [  89 1412]]
[[   0   15]
 [  95 1406]]
[[   0   15]
 [  97 1404]]
[[   0   15]
 [ 118 1383]]
[[   0   15]
 [  77 1424]]
[[   0   15]
 [ 121 1380]]
[[   0   15]
 [ 124 1377]]
[[   0   15]
 [  93 1408]]
[[   0   15]
 [ 108 1393]]
[[   0   15]
 [  98 1403]]
ROC AUC
[0.4703530979347102, 0.46835443037974683, 0.46768820786142573, 0.46069287141905396, 0.4743504330446369, 0.4596935376415723, 0.4586942038640906, 0.46902065289806794, 0.4640239840106596, 0.46735509660226515]
Specificities
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
min_roc_auc = 0.4743504330446369


In [15]:
# Varier le seuil de décision pour voir l'impact
for i in np.arange(0.1, 1., 0.1):
    print("threshold = "+str(i))
    # Instanciation, fit and predict
    func_IFD = IFD.IsolationForest(sample_size = max_samples, n_trees=n_trees)
    func_IFD.fit(X_brut)
    IFD_y_pred_IF, IFD_scores = func_IFD.predict(X=X_brut, threshold=i)
    # Matrice de confusion
    #ttn, tfp, tfn, ttp = confusion_matrix(y_transform, IFD_y_pred_IF).ravel()
    print(confusion_matrix(y_transform, IFD_y_pred_IF))

threshold = 0.1
[[  15    0]
 [1501    0]]
threshold = 0.2
[[  15    0]
 [1501    0]]
threshold = 0.30000000000000004
[[  15    0]
 [1501    0]]
threshold = 0.4
[[  15    0]
 [1501    0]]
threshold = 0.5
[[  15    0]
 [1501    0]]
threshold = 0.6
[[   0   15]
 [ 159 1342]]
threshold = 0.7000000000000001
[[   0   15]
 [   0 1501]]
threshold = 0.8
[[   0   15]
 [   0 1501]]
threshold = 0.9
[[   0   15]
 [   0 1501]]


In [11]:
(119/(119+1382))*100

7.928047968021319

### Execution Of EIF on dataset

In [16]:
import eif
X_brut_numpy = X_brut.to_numpy()

In [17]:
F1  = eif.iForest(X_brut_numpy, ntrees=n_trees, sample_size=max_samples, ExtensionLevel=0)
S1 = F1.compute_paths(X_in=X_brut_numpy)
P1 = np.where(S1>=threshold,-1,1)

# Matrice de confusion
#ttn, tfp, tfn, ttp = confusion_matrix(y_transform_Shuttle, MS_y_pred_IF_Shuttle).ravel()
print(confusion_matrix(y_transform, P1))

[[   0   15]
 [   0 1501]]


In [18]:
# Varier le seuil de décision pour voir l'impact
for i in np.arange(0.1, 1., 0.1):
    print("threshold = "+str(i))
    # Instanciation, fit and predict
    F1  = eif.iForest(X_brut_numpy, ntrees=n_trees, sample_size=max_samples, ExtensionLevel=0)
    S1 = F1.compute_paths(X_in=X_brut_numpy)
    P1 = np.where(S1>=i,-1,1)
    # Matrice de confusion
    #ttn, tfp, tfn, ttp = confusion_matrix(y_transform, IFD_y_pred_IF).ravel()
    print(confusion_matrix(y_transform, P1))

threshold = 0.1
[[  15    0]
 [1501    0]]
threshold = 0.2
[[  15    0]
 [1501    0]]
threshold = 0.30000000000000004
[[  15    0]
 [1501    0]]
threshold = 0.4
[[  15    0]
 [1501    0]]
threshold = 0.5
[[   0   15]
 [1022  479]]
threshold = 0.6
[[   0   15]
 [   0 1501]]
threshold = 0.7000000000000001
[[   0   15]
 [   0 1501]]
threshold = 0.8
[[   0   15]
 [   0 1501]]
threshold = 0.9
[[   0   15]
 [   0 1501]]


EIF ne détecte pas les anomalies. En fait, EIF considère toutes les données comme étant normales

In [ ]:
roc_auc_IF = []
max_samples_IF = []

for j in range(10, len(X_brut), 10):
    print("max_samples = "+str(j))
    max_samples = j
    # Instanciation, fit and predict
    func_IF = IFD.IsolationForest(sample_size = max_samples, n_trees=n_trees)
    func_IF.fit(X_brut)
    y_pred_IF, scores = func_IF.predict(X=X_brut, threshold=threshold)
    print(confusion_matrix(y_transform, y_pred_IF))

    auc_IF = roc_auc_score(y_transform, y_pred_IF)
    if auc_IF >= min_roc_auc:
        print("auc_IF = "+str(auc_IF))
        roc_auc_IF.append(auc_IF)
        max_samples_IF.append(j)

max_samples = 5
max_samples = 10
max_samples = 15
max_samples = 20
max_samples = 25
max_samples = 30
max_samples = 35
max_samples = 40
max_samples = 45
max_samples = 50
max_samples = 55
max_samples = 60
max_samples = 65
max_samples = 70
max_samples = 75
max_samples = 80
max_samples = 85
max_samples = 90
max_samples = 95
max_samples = 100
max_samples = 105
max_samples = 110
auc_IF = 0.6002664890073285
max_samples = 115
max_samples = 120
max_samples = 125
auc_IF = 0.6735509660226515
max_samples = 130
auc_IF = 0.656562291805463
max_samples = 135
auc_IF = 0.6902065289806796
max_samples = 140
auc_IF = 0.7671552298467688
max_samples = 145
auc_IF = 0.7691538974017322
max_samples = 150
auc_IF = 0.8264490339773485
max_samples = 155
auc_IF = 0.8007994670219853
max_samples = 160
auc_IF = 0.755474128358872
max_samples = 165
auc_IF = 0.868421052631579
max_samples = 170
auc_IF = 0.8634243837441706
max_samples = 175
auc_IF = 0.6824117255163225
max_samples = 180
auc_IF = 0.868421052631579
max_samples 

In [ ]:
visu.plot_2D(max_samples_IF, roc_auc_IF,'Max_samples', 'ROC AUC', "ROC AUC Min = "+str(min_roc_auc), link="b").show()

## Conclusion

IForest ne détecte pas les anomalies et fais de fausses alertes.